In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
from glob import glob
from datetime import datetime, date, time, timedelta
from collections import defaultdict

In [2]:
%config InlineBackend.figure_format = 'svg'

In [3]:
snm_df=pd.read_csv('../data/StationNamesMap.csv') #, index_col='TurnstileStationName')
snm_df.head()

,TurnstileStationName,StationLocationName
0,1 AV,1st Av
1,103 ST,103rd St
2,103 ST-CORONA,103rd St
3,104 ST,104th St-102nd St
4,110 ST,110th St


In [4]:
lat_lon_df = pd.read_csv('station_lat_lon.csv')
lat_lon_df.head()

,Station_Name,Station_Latitude,Station_Longitude
0,Ditmars Blvd,40.775036,-73.912034
1,Astoria Blvd-Hoyt Av,40.770258,-73.917843
2,30 Av-Grand Av,40.766779,-73.921479
3,Broadway,40.761820,-73.925508
4,36 Av-Washington Av,40.756804,-73.929575


In [5]:
# Borrowed from Metis class, generates integers.
def generate_numbers():
    """
    An infinite number generator ... stolen from Metis
    """
    x = 0
    while True:
        x += 1
        yield x # instead of return, I use yield, which makes this into a generator!

id_gen = generate_numbers()

In [6]:
# Set the number of rows to display when viewing dataframes / head / tail
pd.set_option("display.max_rows", 101)

In [7]:
# A function for viewing problematic rows. Show preceeding and following rows.
def sample(frame, index):
    """Returns three lines from a dataframe centered around the index parameter"""
    return frame.loc[index-1:index+1]

In [8]:
def am_pm(datetime_series):
    """Creates a Pandas Series object with the same index as the input index and values of
    AM or PM depeneding on the supplied timestamp."""
    s = pd.Series(index=datetime_series.index)
    for index, value in datetime_series.items():
        entry_time = datetime_series[index].time()
        if  entry_time > time(12,0,0) or entry_time == time(0,0,0):
            s[index] = 'PM'
        else:
            s[index] = 'AM'
    return s

In [9]:
def simplify_id(series):
    """Takes a series and changes the values to simple integers. This is to speed up comparisons"""
    s = pd.Series(index=series.index)
    id_set = set(series)
    d = {val : next(id_gen) for val in id_set}
    for index, value in series.iteritems():
        s[index] = d[value]
    return s

In [11]:
# Read data files
# list_of_data_files = sorted(glob('../data/*.txt'))
# list_of_data_files = glob('../data/turnstile_190323.txt')
# list_of_data_files += glob('../data/turnstile_190330.txt')
list_of_data_files = glob('../data/turnstile_190316.txt')
# print(list_of_data_files)

# Read data files into pandas dataframes
list_of_dataframes = [pd.read_csv(file) for file in list_of_data_files]
print(list_of_data_files)
df = pd.concat(list_of_dataframes)
print(df.shape)
df.head()

['../data/turnstile_190316.txt']
(201792, 11)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/09/2019,03:00:00,REGULAR,6972000,2365115
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/09/2019,07:00:00,REGULAR,6972010,2365118
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/09/2019,11:00:00,REGULAR,6972049,2365173
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/09/2019,15:00:00,REGULAR,6972214,2365204
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/09/2019,19:00:00,REGULAR,6972474,2365272


In [12]:
df = df[df['STATION'].str.match(r'TIMES')]
num_timessq_entries = df.shape[0]
print(num_timessq_entries)

2066


In [13]:
df = df.merge(snm_df, how='inner', left_on='STATION', right_on='TurnstileStationName')
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TurnstileStationName,StationLocationName
0,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,03/09/2019,03:00:00,REGULAR,8630701,10554882,TIMES SQ-42 ST,Times Square-42nd St
1,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,03/09/2019,07:00:00,REGULAR,8630752,10554936,TIMES SQ-42 ST,Times Square-42nd St
2,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,03/09/2019,11:00:00,REGULAR,8630929,10555445,TIMES SQ-42 ST,Times Square-42nd St
3,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,03/09/2019,15:00:00,REGULAR,8631253,10556119,TIMES SQ-42 ST,Times Square-42nd St
4,A021,R032,01-00-00,TIMES SQ-42 ST,ACENQRS1237W,BMT,03/09/2019,19:00:00,REGULAR,8631760,10556788,TIMES SQ-42 ST,Times Square-42nd St


In [14]:
df['STATION'] = df['StationLocationName']

In [15]:
df = df.merge(lat_lon_df, how='inner', left_on='STATION', right_on='Station_Name')
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,TurnstileStationName,StationLocationName,Station_Name,Station_Latitude,Station_Longitude
0,A021,R032,01-00-00,Times Square-42nd St,ACENQRS1237W,BMT,03/09/2019,03:00:00,REGULAR,8630701,10554882,TIMES SQ-42 ST,Times Square-42nd St,Times Square-42nd St,40.754672,-73.986754
1,A021,R032,01-00-00,Times Square-42nd St,ACENQRS1237W,BMT,03/09/2019,07:00:00,REGULAR,8630752,10554936,TIMES SQ-42 ST,Times Square-42nd St,Times Square-42nd St,40.754672,-73.986754
2,A021,R032,01-00-00,Times Square-42nd St,ACENQRS1237W,BMT,03/09/2019,11:00:00,REGULAR,8630929,10555445,TIMES SQ-42 ST,Times Square-42nd St,Times Square-42nd St,40.754672,-73.986754
3,A021,R032,01-00-00,Times Square-42nd St,ACENQRS1237W,BMT,03/09/2019,15:00:00,REGULAR,8631253,10556119,TIMES SQ-42 ST,Times Square-42nd St,Times Square-42nd St,40.754672,-73.986754
4,A021,R032,01-00-00,Times Square-42nd St,ACENQRS1237W,BMT,03/09/2019,19:00:00,REGULAR,8631760,10556788,TIMES SQ-42 ST,Times Square-42nd St,Times Square-42nd St,40.754672,-73.986754


In [ ]:
df.drop(['TurnstileStationName', 'StationLocationName', 'Station_Name'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('./notrack/agg_station_data.csv', index=False)

In [ ]:
# Clean up strange spaces in column names
df.columns = df.columns.str.strip()
df.columns

In [ ]:
# Create a tuples of columns 'C/A', 'UNIT', 'SCP', 'STATION'
df['ID'] = pd.Series(zip(df['C/A'], df['UNIT'], df['SCP'], df['STATION']))

In [ ]:
# Replace IDs created above with itengers because processing tuples is really slow
# df['NEW_ID'] = simplify_id(df['ID'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Filter out df['DESC'] == 'RECOVR AUD'. These entries create large jumps in entry counts
df = df[df['DESC'] != 'RECOVR AUD']

In [ ]:
# Add a weekday column
df['DATE_TIME'] = df['DATE'] + ' ' + df['TIME']
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
df['WEEKDAY'] = df['DATE_TIME'].dt.day_name()

In [ ]:
# Drop unneeded columns
df.drop(['LINENAME', 'DIVISION', 'DESC', 'C/A', 'UNIT', 'SCP', 'TIME'], inplace=True, axis=1)

In [ ]:
# Limit to one station for simplicity
# df = df[df['STATION'] == '59 ST']
df.head()

In [ ]:
# cache the number of rows for a future sanity check
rows_in_orig = df.shape[0]

In [ ]:
# Create a disctiondary of dataframes for each turnstile id for entry count processing
# dict_of_ids = {id : df[df['NEW_ID'] == id] for id in df['NEW_ID'].unique()}
dict_of_ids = {id : df[df['ID'] == id] for id in df['ID'].unique()}

In [ ]:
# Sanity check to be sure things worked
# dict_of_ids[8210.0].head()

In [ ]:
# Create Entry counts based on subtracting a an entry count from the entry count in the next row.
for key, frame in dict_of_ids.items():
    dict_of_ids[key]['ENTRY_COUNT'] = abs(frame['ENTRIES'] - frame['ENTRIES'].shift(1)) + \
                                        abs(frame['EXITS'] - frame['EXITS'].shift(1))
    dict_of_ids[key]['ENTRY_COUNT'].fillna(0, inplace=True)

In [ ]:
# Recombine the dataframe from the dictionary of dataframes, now with entry counts
df = pd.concat(dict_of_ids.values())

In [ ]:
# Check the number of rows to be sure we didn't gain or lose anything
assert(df.shape[0] == rows_in_orig)

In [ ]:
df.shape

In [ ]:
# Filter out counts larger than 300 because they are almost always turnstile resets.
df = df[df['ENTRY_COUNT'] < 3000]

In [ ]:
df.shape

In [ ]:
# Create a DATE column from the DATE_TIME timestamp.
df['DATE'] = df['DATE_TIME'].dt.date

In [ ]:
df.head()

In [ ]:
df[['ENTRY_COUNT']].describe()

In [ ]:
# Sum up entry counts by date and weekday. This is a little redundant, but the WEEKDAY
# column is needed for later processing.
# gp_date_weekday = df.groupby(['STATION', 'DATE', 'WEEKDAY'], as_index=False).sum()
gp_date_weekday = df.groupby(['STATION', 'DATE'], as_index=False).sum()

In [ ]:
gp_date_weekday.head()

In [ ]:
# Calculate the mean number of entries for each station.
daily_mean_by_station = gp_date_weekday.groupby('STATION', as_index=False).mean()

In [ ]:
# Round off the mean values as they have a crazy number of decimal places.
daily_mean_by_station['ENTRY_COUNT'] = daily_mean_by_station['ENTRY_COUNT'].round(0)
# daily_mean_by_station.sort_values('ENTRY_COUNT', ascending=False, inplace=True)

In [ ]:
# Create a new dataframe of the file largest stations by entry count
five_largest = daily_mean_by_station.nlargest(5, 'ENTRY_COUNT').sort_values('ENTRY_COUNT', ascending=False)
station = five_largest.STATION
entry_counts = five_largest.ENTRY_COUNT
five_largest.to_csv('five_largest_stations.csv')


In [ ]:
five_largest.head()

In [ ]:
five_largest.head()

In [ ]:
# Create a reversed index so that stations display in descending order in the bar graph
new_index = list(reversed(station.index))

In [ ]:
# Plot the bar chart of the top 5 stations
plt.barh(station[new_index], entry_counts[new_index]);
plt.xlabel('Passenger Count', fontsize = 14, weight = 'bold',color='black');
plt.title('Top Five Stations by Passenger Count', fontsize = 15, weight = 'bold',color='black')
plt.savefig('top_five_stations_barh.png')

In [ ]:
# Create a series of the top ten stations
top_ten_stations = daily_mean_by_station.nlargest(10, 'ENTRY_COUNT').STATION
top_ten_stations

In [ ]:
top_ten_series = pd.Series(gp_top_ten_weekday.STATION.unique())[[2, 9, 7, 3, 1, 4, 0, 8, 6, 5]]

In [ ]:
top_ten_df = gp_date_weekday[gp_date_weekday['STATION'].isin(top_ten_stations)]
gp_top_ten_weekday = top_ten_df.groupby(['STATION', 'WEEKDAY'], as_index=False).mean()
# gp_top_ten_weekday.drop(['ENTRIES', 'NEW_ID'], axis=1, inplace=True)
gp_top_ten_weekday.drop(['ENTRIES'], axis=1, inplace=True)
gp_top_ten_weekday.head(2)
test = gp_top_ten_weekday.copy()
test.set_index('WEEKDAY', inplace=True)
wd_idx = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
gp_top_ten_weekday = test.copy()
legend_names = []
plt.figure(figsize = [12,5])
# for station in gp_top_ten_weekday.STATION.unique():
for station in top_ten_series:
    tmp_df = gp_top_ten_weekday[gp_top_ten_weekday['STATION'] == station]
    plt.plot(tmp_df['ENTRY_COUNT'][wd_idx])
    legend_names.append(station)
plt.ylabel('Passenger Count', fontsize = 14, weight = 'bold',color='black');
plt.title('Top Stations by Passenger Count', fontsize = 16, weight = 'bold',color='black')
plt.xlabel('Week Day', fontsize = 14, weight = 'bold',color='black')
# plt.legend(legend_names, shadow=True, loc = 0, fontsize=8);
plt.legend(top_ten_series, shadow=True, loc = 0, fontsize=8);
plt.savefig('top_stations.png')

In [ ]:
# gp_top_ten_weekday.head(20)

***

In [ ]:
tf = df[df['STATION'] == '34th St']
tf.shape

In [ ]:
tf.head(20)

In [ ]:
pickle.dump(df, open( "NYC_TS_DF.pickle", "wb" ) )

In [ ]:
gp_top_ten_weekday.STATION.unique()

In [ ]:
len(df.STATION.unique())

In [ ]:
df['ENTRY_COUNTS'] = count_entries(df)

In [ ]:
df['PCT_RANK'] = (df['ENTRY_COUNTS'].rank(pct=True) * 100).round(0)

In [ ]:
df.head()

In [ ]:
df[df['PCT_RANK'] == 99].shape

In [ ]:
# Filter out ENTRY_COUNTS greater than 3000 because these are usually counter resets
df = df[df['ENTRY_COUNTS'] < 3000]

In [ ]:
df[df['ENTRY_COUNTS'] < 0]

In [ ]:
print(df['ENTRY_COUNTS'].min(), df['ENTRY_COUNTS'].max()) 

In [ ]:
df['AM/PM'] = am_pm(df['DATE_TIME'])

In [ ]:
df.head()

In [ ]:
gp = df.groupby(['STATION', 'WEEKDAY', 'AM/PM'], as_index=False).sum()

In [ ]:
gp.head(100)

In [ ]:
gp_sorted = gp.sort_values(['ENTRY_COUNTS'], ascending=False)

In [ ]:
gp_sorted.nlargest(20, 'ENTRY_COUNTS')

In [ ]:
plt.hist(df.ENTRY_COUNTS[df.ENTRY_COUNTS < 2000])

In [ ]:
df = df[df['ENTRY_COUNTS'] < 4000]

In [ ]:
df.loc[36217:36221]

In [ ]:
df.ENTRY_COUNTS.median()

In [ ]:
df.ENTRY_COUNTS.mean()

In [ ]:
df.head()

In [ ]:
# Create a dict dataframes that contain the turnstile data for each station
dict_of_station_frames = {station : df[df.STATION == station] for station in df['STATION'].unique()}
len(dict_of_station_frames.keys())

In [ ]:
dict_of_station_frames['59 ST'].head()

In [ ]:
for name in dict_of_station_frames.keys():
    print(name + ':\n')
    print(dict_of_station_frames[name].describe())

In [ ]:
station_counts = defaultdict(int)
for name in dict_of_station_frames.keys():
    frame = dict_of_station_frames[name].copy()
    frame['ENTRY_COUNT'] = frame['ENTRIES'] - frame['ENTRIES'].shift(1)
    frame['ENTRY_COUNT'].fillna(0, inplace=True)
    dict_of_station_frames[name] = frame
    print(dict_of_station_frames[name].head(5))
    station_counts[name] = dict_of_station_frames[name]['ENTRY_COUNT'].sum()

In [ ]:
for key, value in station_counts.items():
    print(key, ':', value)

In [ ]:
for name in dict_of_station_frames.keys():
    

In [ ]:
df.head()

In [ ]:
# Filter data to daylight hours
times = ['12:00:00', '16:00:00', '20:00:00']
df = df[df['TIME'].isin(times)]

In [ ]:
gd = df.groupby(['STATION', 'WEEKDAY', 'TIME'], as_index=False).count()

In [ ]:
gd.head()